# Projeto 2 - Ciência dos Dados

Nome: Ellen Beatriz Shen

Nome: Gabriela Moreno Boriero

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [ ]:
import math
import os.path
import pandas as pd
import json
from random import shuffle
import re 

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
#Leitura do excel para treinar o código
tudo = pd.read_excel("base_acai.xlsx", sheet_name="Treinamento")

#Série de todos os dados + sua tabela absoluta
serie_tudo = pd.Series(tudo.Treinamento)
tabela_tudo_absoluta = serie_tudo.value_counts()

In [ ]:
#Função para limpar os sinais básicos.
def limpeza(texto):
    import string
    pontuacao = '[!-.:?;]'
    padrao = re.compile(pontuacao)
    text_subbed = re.sub(pattern, ' ', texto)
    return text_subbed

In [ ]:
#
relevante_acai = limpeza((open("relevante.txt", "r", encoding="utf8").read()).lower())
irrelevante_acai = limpeza((open("irrelevante.txt", "r", encoding="utf8").read()).lower())

In [ ]:
relevante_relativa = (pd.Series(relevante_acai.split())).value_counts(True)
relevante_absoluta = (pd.Series(relevante_acai.split())).value_counts(False)

irrelevante_relativa = (pd.Series(irrelevante_acai.split())).value_counts(True)
irrelevante_absoluta = (pd.Series(irrelevante_acai.split())).value_counts(False)

In [ ]:
probabilidade_relevante = tabela_relevante.sum()/tabela_tudo_absoluta.sum()
probabilidade_irrelevante = tabela_irrelevante.sum()/tabela_tudo_absoluta.sum()

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
def laplace(palavra, tabela1, tabela2):
    if palavra in tabela1:
        x = tabela1[palavra]
    else:
        x = 0
        
    if palavra in tabela2:
        y = tabela2[palavra]
    else:
        y = 0 
    
    prob_relevante = (x + 1)/ (len(tabela1) + len(set(tabela1+tabela2)))
    prob_irrelevante = (y + 1)/ (len(tabela2) + len(set(tabela1+tabela2)))
    return [prob_relevante, prob_irrelevante]    

In [ ]:
base_teste = pd.read_excel("base_acai.xlsx",sheet_name="Teste")
series_tweets = pd.Series(list(base_teste.Teste))

In [ ]:
tweets_limpos = []
for tweet in series_tweets:
    tweets_limpos.append(cleanup(tweet.lower()))

    
classificacao = {}
lista_classificador=[]
for frase in tweets_limpos:
    palavras_da_frase = list(frase.split())
    relevante = 1
    irrelevante = 1
    
    for palavra in palavras_da_frase:

        if palavra in relevante_relativa and \
        palavra in irrelevante_relativa:
            relevante *= relevante_relativa[palavra]
            irrelevante *= irrelevante_relativa[palavra]
            
        elif palavra in relevante_relativa and \
        palavra not in irrelevante_relativa:
            relevante *= relevante_relativa[palavra]
            irrelevante *= laplace(palavra,relevante_relativa,irrelevante_relativa)[1]
        
        elif palavra in irrelevante_relativa and \
        palavra not in relevante_relativa:
            relevante *= laplace(palavra,relevante_relativa,irrelevante_relativa)[0]
            irrelevante *= irrelevante_relativa[palavra]
            
        else:
            relevante *= laplace(palavra,relevante_relativa,irrelevante_relativa)[0]
            irrelevante *= laplace(palavra,relevante_relativa,irrelevante_relativa)[1]
            
    prob_relevante = probabilidade_relevante*relevante
    prob_irrelevante = probabilidade_irrelevante*irrelevante
    
    if prob_relevante > prob_irrelevante:
        classificacao[frase] = 0
        lista_classificador.append(0)
    else:
        classificacao[frase] = 1
        lista_classificador.append(1)
       
classificados = pd.Series(classificacao)

In [ ]:
#comparando o teste com a classificação 
for tweet in classificacao:
    dic={
        'Tweet': tweets_limpos,
        'Resultafo Teste': lista_classificador,
        'Classificação': base_teste["Classe"]
    }
tabela_classifiação= pd.DataFrame(dic, columns=['Tweet','Resultafo Teste','Classificação'])

In [ ]:
certos=0
errados=0

for teste,certo in zip(lista_classificador,base_teste["Classe"]):
    if teste==certo:
        certos+=1
    else:
        errados+=1
       
porcentagem_certos=(certos/299)*100
porcentagem_errados=(errados/299)*100
print("Porcentagem de acertos= {}%".format(porcentagem_certos))
print("Porcentagem de erros= {}%".format(porcentagem_errados))

In [ ]:
verdadeiros_positivo=0
falsos_positivos=0
verdadeiros_negativos=0
falsos_negativos=0
#relevante=0 e irrelevante=1
for resultado_teste,resultado_certo in zip(lista_classificador,base_teste["Classe"]):
    if resultado_certo==0 and resultado_teste==0:
        verdadeiros_positivo+=1
    elif resultado_certo==1 and resultado_teste==0:
        falsos_positivos+=1
    elif resultado_certo==1 and resultado_teste==1:
        verdadeiros_negativos+=1
    elif falsos_negativos==0 and resultado_teste==1:
        falsos_negativos+=1
    
Porcentagem_verdadeiros_positivo=(verdadeiros_positivo/229)*100
Porcentagem_falsos_positivos=(falsos_positivos/229)*100
Porcentagem_verdadeiros_negativos=(verdadeiros_negativos/229)*100
Porcentagem_falsos_negativos=(falsos_negativos/229)*100

print("Porcentagem de verdadeiros positivos: {}%".format(Porcentagem_verdadeiros_positivo))
print("Porcentagem de falsos positivos: {}%".format(Porcentagem_falsos_positivos))
print("Porcentagem de verdadeiros negativos: {}%".format(Porcentagem_verdadeiros_negativos))
print("Porcentagem de falsos negativos: {}%".format(Porcentagem_falsos_negativos))



___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**